In [ ]:
# mms_mvab_cl.ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Latex

import os

In [ ]:
from numpy import linalg as LA
# #########################
# #GET MAGNETIC FIELD DATA#
# #########################
def mms_mvab_cl(m1,m2,mag_arr_str):
    mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
    m1 = pd.to_datetime(m1)
    m2 = pd.to_datetime(m2)
    # ##########################################################
    # Check for NaN values
    NaNs = mag_arr.isnull().values.any()
    if NaNs == True:
        print,'Nans were found: '
        print, mag_arr.isnull().values.any()
        print,'Replacing with average of surrounding data pts '
        # Indices of NaNs in each component
        NaNs_Bx = np.argwhere(np.isnan(bx))
        NaNs_By = np.argwhere(np.isnan(by))
        NaNs_Bz = np.argwhere(np.isnan(bz))

        # If there are NaNs, replace with the average of surrounding couple of data points
        for i in range(len(NaNs_Bx)):
            bx[NaNs_Bx[i]] = np.nanmean(bx[NaNs_Bx[i-3]:NaNs_Bx[i+3]])
            by[NaNs_By[i]] = np.nanmean(by[NaNs_By[i-3]:NaNs_By[i+3]])
            bz[NaNs_Bz[i]] = np.nanmean(bz[NaNs_Bz[i-3]:NaNs_Bz[i+3]])                           
    time = np.array(mag_arr['times'])
    bx = np.array(pd.to_numeric(mag_arr['Bx']))
    by = np.array(pd.to_numeric(mag_arr['By']))
    bz = np.array(pd.to_numeric(mag_arr['Bz']))
    n1 = mag_arr.date[0]
    n2 = mag_arr.date[len(mag_arr.date)-1]
    if m1 < n1:
        print('WARNING!!! start time of current sheet =',m1, 'start time of B-field data =', n1)
    if m1 > n2:
        print('WARNING!!! start time of current sheet =',m1,'end time of B-field data =',n2)
    if m2 > n2:
        print('WARNING!!! end time of current sheet =',m2,'end time of B-field data =',n2)
    if m2 < n1:
        print('WARNING!!! end time of current sheet =',m2,'start time of B-field data =',n1)
    v1 = mag_arr.date.searchsorted(m1)
    v2 = mag_arr.date.searchsorted(m2)
    bx = bx[v1:v2]
    by = by[v1:v2]
    bz = bz[v1:v2]
    
    
    # create the minimum variance matrix. 
    mm = np.zeros([3, 3])
    bxa = np.mean(bx)
    bya = np.mean(by)
    bza = np.mean(bz)
    mm[0,0] = np.mean(bx*bx) - bxa*bxa
    mm[1,0] = np.mean(bx*by) - bxa*bya
    mm[2,0] = np.mean(bx*bz) - bxa*bza
    mm[0,1] = mm[1,0]
    mm[1,1] = np.mean(by*by) - bya*bya
    mm[2,1] = np.mean(by*bz) - bya*bza
    mm[0,2] = mm[2,0]
    mm[1,2] = mm[2,1]
    mm[2,2] = np.mean(bz*bz) - bza*bza
    
    # The 3 eigenvectors represent directions of maximum, intermediate, and minimum variance of the field
    # component along each vector. 
    
    # The corresponding eigenvalues (λ) represent the actual variances in those field components & are non-negative 
    
    
    # The eigenvector x3 (corresponding to the smallest eigenvalue λ3) is used as the estimator for 
    # the vector normal to the current sheet.
    
    # λ3 represents the variance of the magnetic field component along the estimated normal. 
    
    # The eigenvectors x1 and x2, corresponding to maximum and intermediate variance
    # are tangential to the transition layer
    
    
    
    # eigen vectors should be in GSE coordinate system right?
    eigenvalues, eigenvectors = LA.eig(mm)    
    idx = np.argsort(eigenvalues)
    E_vals_sorted = eigenvalues[idx]
    E_vec_sorted = eigenvectors[:,idx]

    return E_vals_sorted,E_vec_sorted